In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
okt = Okt()

In [2]:
train_data = pd.read_csv('dataset/MOVIEREV_DATA_TABLE.csv')

In [3]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['MCONTENT'] = train_data['MCONTENT'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [4]:
stopword=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자']

In [5]:
def tokenizer(raw,pos=["Noun","Alpha","Verb","Number"]):
    return [
        word for word, tag in okt.pos(raw,norm=True,stem=True)
        if len(word) > 1 and tag in pos and word not in stopword
    ]

In [6]:
tfidf = TfidfVectorizer(tokenizer=tokenizer)
tfidf_matrix = tfidf.fit_transform(train_data['MCONTENT'])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix , tfidf_matrix)

In [8]:
cosine_sim

array([[1.        , 0.03822345, 0.        , ..., 0.        , 0.        ,
        0.02001549],
       [0.03822345, 1.        , 0.        , ..., 0.        , 0.        ,
        0.02303219],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.02001549, 0.02303219, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [9]:
indexdata = pd.Series(train_data.index,index=train_data['MTITLE']).drop_duplicates()

In [10]:
indexdata.head(10)

MTITLE
정직한 후보           0
다만 악에서 구하소서      1
오케이 마담           2
오케이 마담           3
강철비2: 정상회담       4
들어는 봤니? 모건 부부    5
반도               6
강철비2: 정상회담       7
다만 악에서 구하소서      8
콰이어트 플레이스        9
dtype: int64

In [11]:
idx = indexdata['크로우']

In [12]:
print(idx)

6628


In [22]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = pd.Series(indexdata[title])
    
    idx.index = range(len(idx))
    sim_scores = list(enumerate(cosine_sim[idx[0]]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #print(sim_scores)
    #print('-'*35)
    # 가장 유사도가 높은 10 개만 추출 
    sim_scores = sim_scores[0:11]
    # 인덱스 구하기
    movie_indices = [i[0] for i in sim_scores]
    # 구한 인덱스를 기반으로 데이터셋에서 찾아서 반환 
    return train_data[['MTITLE','MCONTENT']].iloc[movie_indices]

In [21]:
get_recommendations('크로우')

,MTITLE,MCONTENT
6628,크로우,음울한 간지 그 자체 브루스 리의 의문의 죽음과 그의 아들까지 의문의 죽음으로 영화...
4376,인베이젼,몇 번을 봐도 의문점만 가득하다
8352,인셉션,꿈일까 현실일까 아직도 남는 의문
9605,다만 악에서 구하소서,이게 점대라고 보는 내내 이 영화를 왜 만들었을까 라는 의문만 가득했다이 영화로 득...
797,더 홈즈맨,커디의 죽음이 너무 갑작스럽고 허망하다고 심리묘사가 부족해 좀 더 이해될수 있도록 ...
3442,업그레이드,주인공 빼고 다 죽음 잔인함 청불영화
5953,익스트랙션,주인공은 살짜리 아들이 죽고나서 하루하루가 지옥이었던 삶을 살았다가 끝에서 주인공이...
741,사도,애증에서 시작되어 감정의 골이 점점 깊어져가는 부모와 자식간의 갈등은 시대를 불문하...
6954,다만 악에서 구하소서,이정재 눈빛연기는 진짜 전설이다황정민 감정연기는 진짜 전설이다박정민 트젠연기는 진짜...
5247,#살아있다,감독이 뭐 하는 사람인지 궁금합니다 이딴영화가 흥행 성공했다는게 가장 의문 초반엔 ...


In [16]:
# 아래에서 크로운 - 중복되지 않는 데이터, 오케이마담과 같은 타이틀은 중복이 됩니다.
# 이것을 해결한 다음에 카페에 연습문제 게시 바랍니다

In [23]:
get_recommendations('오케이 마담')

,MTITLE,MCONTENT
2,오케이 마담,가족들이랑 보고왔는데 다들 넘 재미있데요
6239,오케이 마담,넘나 재미나게 보고왔어요 보세요 보세요
6594,오케이 마담,넘나 재밌엇어요
6035,강철비2: 정상회담,보고옴 알바라는건 오바 같고 그럭저럭 볼만하더만
7335,강철비2: 정상회담,이런영화눈 가족과함께
6121,반교: 디텐션,기대없이갔다가 너무 재밌게보고왔습니다
1749,다만 악에서 구하소서,배우들 연기만 보고왔음 그냥 저냥 했음
3503,오케이 마담,아무생각 없이 봤다 재밌게 기분좋게 보고왔습니다 ㅎㅎ
3741,오케이 마담,간만에 웃으면서 영화보고왔네용
8413,오케이 마담,재미있네요 가족이나 친구랑 가볍게 보고 오면 좋을 코미디 영화인 것 같아요 어제 보...
